# Iteraatio3 - ominaisuuksien valintaa omalla tavalla

In [1]:
import numpy as np
import pandas as pd

WORKING_DIRECTORY = '/mnt/c/git/masters-thesis-code/jupyter/code/'

## Ladataan yritysdata

In [8]:
SELECTED_COMPANY_FEATURES = ['company_form_code', 'location_municipality_code', 
                             'location_region_code', 'company_status_code', 'industry_code', 'turnover', 
                             'net_profit', 'personnel_average', 'performer_ranking_points', 'risk_rating_class']

In [9]:
COMPANIES_DF = pd \
        .read_csv(WORKING_DIRECTORY + 'data/prod_data_companies_more_data_2021_09_16.csv',
                  delimiter='\t',
                  na_values='(null)',
                  dtype={
                      'business_id': str,
                      'business_id_start_date': str,
                      'company_name': str,
                      'email_address': str,
                      'industry_code': str,
                      'language': str,
                      'company_form': str,
                      'company_form_code': str,
                      'domicile_code': str,
                      'region_code': str,
                      'location_municipality_code': str,
                      'location_region_code': str,
                      'company_status_code': str,
                      'balance_industry_code': str,
                      'personnel_class': str,
                      'turnover_class': str,
                      'statement_date': str,
                      'statement_length': float,
                      'target_year': str,
                      'turnover': float,
                      'raw_materials_use': float,
                      'current_ratio': float,
                      'personnel_expenses': float,
                      'investment_expenses_cash_flow': float,
                      'ebitda': float,
                      'operating_profit_per_turnover': float,
                      'ebit': float,
                      'turnover_growth_percent': float,
                      'ebit_per_turnover': float,
                      'short_term_sales_receivable': float,
                      'net_profit': float,
                      'net_profit_per_turnover': float,
                      'aggregate_equity': float,
                      'return_on_equity_percent': float,
                      'equity_ratio': float,
                      'long_term_sales_receivable': float,
                      'quick_ratio': float,
                      'buildings': float,
                      'return_on_investment_percent': float,
                      'total_assets': float,
                      'external_services': float,
                      'aggregate_current_assets': float,
                      'personnel_average': float,
                      'concern': str,
                      'bulk': str,
                      'performer_ranking_points': float,
                      'risk_rating_class': str,
                      'latest': str
                  }
                  )

COMPANIES_DF.head()

,business_id,business_id_start_date,company_name,email_address,industry_code,language,company_form,company_form_code,domicile_code,region_code,...,return_on_investment_percent,total_assets,external_services,aggregate_current_assets,personnel_average,concern,bulk,performer_ranking_points,risk_rating_class,latest
0,31431209,2020-06-22,Tmi Onerva Miettunen,onerva.adalmiina@gmail.com,43341,CL_1,Yksityinen elinkeinonharjoittaja,CO_26,091,RE_01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,32087307,2021-05-04,Koti Vaihtoon LKV Oy,matti.leskinen@kotivaihtoon.com,68310,CL_1,Osakeyhtiö,CO_16,743,RE_14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,18601103,2003-10-24,Teollisuuden Monialapalvelu T:mi Oksanen,NaN,68209,CL_1,Yksityinen elinkeinonharjoittaja,CO_26,179,RE_13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20469041,2006-07-04,POLYBEAT,NaN,74909,CL_2,Kuolinpesä,CO_53,091,RE_01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,24844507,2012-06-18,SAHOJEN PUU,kalle.kantola@hasa.fi,NaN,CL_1,Yksityinen elinkeinonharjoittaja,CO_26,069,RE_17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Käsitellään konsernit (prefiksoi)

In [10]:
def handle_concerns(row):
    if (row.concern == 'true'):
        return 'K-' + row.business_id
    return row.business_id

COMPANIES_DF['business_id'] = COMPANIES_DF.apply(handle_concerns, axis=1)



## Valitaan halutut sarakkeet

In [11]:
COMPANIES_DF = COMPANIES_DF[['business_id', 'company_name'] + SELECTED_COMPANY_FEATURES]
COMPANIES_DF.head()

,business_id,company_name,company_form_code,location_municipality_code,location_region_code,company_status_code,industry_code,turnover,net_profit,personnel_average,performer_ranking_points,risk_rating_class
0,31431209,Tmi Onerva Miettunen,CO_26,091,01,AKT,43341,NaN,NaN,NaN,NaN,NaN
1,32087307,Koti Vaihtoon LKV Oy,CO_16,091,01,AKT,68310,NaN,NaN,NaN,NaN,NaN
2,18601103,Teollisuuden Monialapalvelu T:mi Oksanen,CO_26,NaN,NaN,AKT,68209,NaN,NaN,NaN,NaN,NaN
3,20469041,POLYBEAT,CO_53,NaN,NaN,AKT,74909,NaN,NaN,NaN,NaN,NaN
4,24844507,SAHOJEN PUU,CO_26,NaN,NaN,AKT,NaN,NaN,NaN,NaN,NaN,NaN


## Käsitellään toimialakoodit

In [12]:
def transform_industry(row):
    industry_code = row.industry_code
    if (pd.isnull(industry_code)):
        return 'NaN'
    return industry_code[0:2]

COMPANIES_DF['industry_code'] = COMPANIES_DF.apply(transform_industry, axis=1)
COMPANIES_DF['industry_code']

0           43
1           68
2           68
3           74
4          NaN
          ... 
1337863     70
1337864     78
1337865     42
1337866     62
1337867     95
Name: industry_code, Length: 1337868, dtype: object

## Käsitellään riskiluokitus

In [13]:
def transform_risk_rating(row):
    risk_rating = row.risk_rating_class
    if (pd.isnull(risk_rating)):
        return 'NaN'
    return risk_rating.split('_')[0]

COMPANIES_DF['risk_rating_class'] = COMPANIES_DF.apply(transform_risk_rating, axis=1)
COMPANIES_DF

,business_id,company_name,company_form_code,location_municipality_code,location_region_code,company_status_code,industry_code,turnover,net_profit,personnel_average,performer_ranking_points,risk_rating_class
0,31431209,Tmi Onerva Miettunen,CO_26,091,01,AKT,43,NaN,NaN,NaN,NaN,NaN
1,32087307,Koti Vaihtoon LKV Oy,CO_16,091,01,AKT,68,NaN,NaN,NaN,NaN,NaN
2,18601103,Teollisuuden Monialapalvelu T:mi Oksanen,CO_26,NaN,NaN,AKT,68,NaN,NaN,NaN,NaN,NaN
3,20469041,POLYBEAT,CO_53,NaN,NaN,AKT,74,NaN,NaN,NaN,NaN,NaN
4,24844507,SAHOJEN PUU,CO_26,NaN,NaN,AKT,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1337863,27527742,Ingenium Partners Oy,CO_16,NaN,NaN,AKT,70,27000.0,0.0,NaN,26.0,GREEN
1337864,31204145,Rainmaker Voima Oy,CO_16,091,01,AKT,78,2443000.0,283000.0,60.0,62.0,GREEN
1337865,30137091,We Build Parks Finland Oy,CO_16,261,19,AKT,42,NaN,NaN,NaN,NaN,NaN
1337866,32150097,Orankien Osuuskunta,CO_14,091,01,AKT,62,NaN,NaN,NaN,NaN,NaN


## Muutetaan valittujen sarakkeiden tiedot avainsanoiksi

In [14]:
def transform_str_to_keyword(val, col_name):
    if (pd.isnull(val)):
        return col_name + '+' + 'NaN'
    return col_name + '+' + val

boundaries = COMPANIES_DF.quantile([0.2, 0.4, 0.6, 0.8, 0.9, 0.95, 0.98, 0.99])
print(boundaries)
print(COMPANIES_DF.min(axis=0))
print(COMPANIES_DF.max(axis=0))

def transform_num_to_keyword(val, col_name):
    if (pd.isnull(val)):
        return col_name + '+' + 'NaN'

    col_boundaries = []

    if (col_name == 'turnover'):
        col_boundaries = [20000, 100000, 250000, 1000000, 10000000]

    if (col_name == 'net_profit'):
        col_boundaries = [0, 10000, 50000, 100000, 250000, 1000000]

    if (col_name == 'personnel_average'):
        col_boundaries = [5, 20, 100, 300]

    if (col_name == 'performer_ranking_points'):
        col_boundaries = [30, 50, 70, 85]

    
    for index, item in enumerate(col_boundaries):
        if (val < item):
            return col_name + '+' + str(index)
        
    return col_name + '+' + 'top'

        turnover  net_profit  personnel_average  performer_ranking_points
0.20     22000.0     -4000.0               1.00                      31.0
0.40     87000.0         0.0               2.00                      46.0
0.60    234000.0      8000.0               5.00                      56.0
0.80    825000.0     43000.0              16.00                      70.0
0.90   2390000.0    125000.0              41.00                      77.0
0.95   6721200.0    305000.0              95.95                      82.0
0.98  23293680.0    950780.0             294.38                      86.0
0.99  54267840.0   2244000.0             599.57                      88.0


/tmp/ipykernel_142/3799369724.py:8: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print(COMPANIES_DF.min(axis=0))


business_id                     00211027
company_status_code                  AKT
industry_code                         00
turnover                      -2998000.0
net_profit                 -1998200000.0
personnel_average                    0.0
performer_ranking_points             0.0
risk_rating_class                  GREEN
dtype: object
business_id                    K-92105327
company_status_code               SELTILA
industry_code                         NaN
turnover                    49015000000.0
net_profit                   1515000000.0
personnel_average                 92039.0
performer_ranking_points             98.0
risk_rating_class                  YELLOW
dtype: object


/tmp/ipykernel_142/3799369724.py:9: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print(COMPANIES_DF.max(axis=0))


In [15]:
for col_name in SELECTED_COMPANY_FEATURES:
    dtype = COMPANIES_DF[col_name].dtypes
    if dtype == 'object':
        COMPANIES_DF[col_name] = COMPANIES_DF[col_name].apply(lambda x: transform_str_to_keyword(x, col_name))
    if dtype == 'float64' or dtype == 'int64':
        COMPANIES_DF[col_name] = COMPANIES_DF[col_name].apply(lambda x: transform_num_to_keyword(x, col_name))
        
COMPANIES_DF

,business_id,company_name,company_form_code,location_municipality_code,location_region_code,company_status_code,industry_code,turnover,net_profit,personnel_average,performer_ranking_points,risk_rating_class
0,31431209,Tmi Onerva Miettunen,company_form_code+CO_26,location_municipality_code+091,location_region_code+01,company_status_code+AKT,industry_code+43,turnover+NaN,net_profit+NaN,personnel_average+NaN,performer_ranking_points+NaN,risk_rating_class+NaN
1,32087307,Koti Vaihtoon LKV Oy,company_form_code+CO_16,location_municipality_code+091,location_region_code+01,company_status_code+AKT,industry_code+68,turnover+NaN,net_profit+NaN,personnel_average+NaN,performer_ranking_points+NaN,risk_rating_class+NaN
2,18601103,Teollisuuden Monialapalvelu T:mi Oksanen,company_form_code+CO_26,location_municipality_code+NaN,location_region_code+NaN,company_status_code+AKT,industry_code+68,turnover+NaN,net_profit+NaN,personnel_average+NaN,performer_ranking_points+NaN,risk_rating_class+NaN
3,20469041,POLYBEAT,company_form_code+CO_53,location_municipality_code+NaN,location_region_code+NaN,company_status_code+AKT,industry_code+74,turnover+NaN,net_profit+NaN,personnel_average+NaN,performer_ranking_points+NaN,risk_rating_class+NaN
4,24844507,SAHOJEN PUU,company_form_code+CO_26,location_municipality_code+NaN,location_region_code+NaN,company_status_code+AKT,industry_code+NaN,turnover+NaN,net_profit+NaN,personnel_average+NaN,performer_ranking_points+NaN,risk_rating_class+NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1337863,27527742,Ingenium Partners Oy,company_form_code+CO_16,location_municipality_code+NaN,location_region_code+NaN,company_status_code+AKT,industry_code+70,turnover+1,net_profit+1,personnel_average+NaN,performer_ranking_points+0,risk_rating_class+GREEN
1337864,31204145,Rainmaker Voima Oy,company_form_code+CO_16,location_municipality_code+091,location_region_code+01,company_status_code+AKT,industry_code+78,turnover+4,net_profit+5,personnel_average+2,performer_ranking_points+2,risk_rating_class+GREEN
1337865,30137091,We Build Parks Finland Oy,company_form_code+CO_16,location_municipality_code+261,location_region_code+19,company_status_code+AKT,industry_code+42,turnover+NaN,net_profit+NaN,personnel_average+NaN,performer_ranking_points+NaN,risk_rating_class+NaN
1337866,32150097,Orankien Osuuskunta,company_form_code+CO_14,location_municipality_code+091,location_region_code+01,company_status_code+AKT,industry_code+62,turnover+NaN,net_profit+NaN,personnel_average+NaN,performer_ranking_points+NaN,risk_rating_class+NaN


In [16]:
ITEM_IDS = list(COMPANIES_DF['business_id'].unique())

## Ladataan vuorovaikutusdata

In [17]:
interactions_tmp = pd \
    .read_csv(WORKING_DIRECTORY + 'data/interactions_2021_08_19.csv',
             delimiter='\t',
             dtype={
                 'group_id': 'string',
                 'business_id': 'string',
                 'owner': 'string'
             })

# otetaan pois 1 kokoiset ryhmät
group_sizes = interactions_tmp['group_id'].value_counts()
group_sizes_df = pd.DataFrame({'group_id': group_sizes.index, 'group_size': group_sizes.values})

INTERACTIONS_WITH_GROUP_SIZES_DF = interactions_tmp.merge(group_sizes_df, on='group_id')             
interactions_tmp = INTERACTIONS_WITH_GROUP_SIZES_DF[INTERACTIONS_WITH_GROUP_SIZES_DF.group_size >= 2]
#interactions_tmp = INTERACTIONS_WITH_GROUP_SIZES_DF[INTERACTIONS_WITH_GROUP_SIZES_DF.group_size <= 3000]
interactions_tmp.sort_values('group_size')

,group_id,business_id,owner,group_size
155696,3e9dd356-2b21-45ae-9ee4-7cd6cc122fe1,07577937,5e87095492119e00066e7158,2
106198,31503959-943a-4081-abcc-dc80e5cb0402,15093748,5db034c64320cd0006d2b788,2
313746,cab22fae-db47-46b6-b902-3d9a1b1051f6,01163004,5e4534bc7bf061000697e940,2
313747,cab22fae-db47-46b6-b902-3d9a1b1051f6,10410900,5e4534bc7bf061000697e940,2
545392,0967d6ed-88b7-4023-a720-f09f7051f24d,17944788,5efdbc656488210007bc27f6,2
...,...,...,...,...
8042,a5c6ce2e-22ab-4871-bd72-e5da294b33cc,16029641,5e1489f3c2f568000654ecbb,3999
8043,a5c6ce2e-22ab-4871-bd72-e5da294b33cc,16030167,5e1489f3c2f568000654ecbb,3999
8044,a5c6ce2e-22ab-4871-bd72-e5da294b33cc,16030415,5e1489f3c2f568000654ecbb,3999
8031,a5c6ce2e-22ab-4871-bd72-e5da294b33cc,16001948,5e1489f3c2f568000654ecbb,3999


In [18]:
# lisätään konserniyrityksille interaktiot
concern_interactions = interactions_tmp.copy()
concern_interactions['business_id'] = 'K-' + concern_interactions['business_id'].astype(str)
concern_interactions = concern_interactions[concern_interactions.business_id.isin(ITEM_IDS)]
concern_interactions

,group_id,business_id,owner,group_size
5,c2626398-faac-4ff3-b02d-cdc64b50cdaa,K-01681709,60646431ae18cb00063ed63f,1862
6,c2626398-faac-4ff3-b02d-cdc64b50cdaa,K-15055514,60646431ae18cb00063ed63f,1862
7,c2626398-faac-4ff3-b02d-cdc64b50cdaa,K-01876143,60646431ae18cb00063ed63f,1862
9,c2626398-faac-4ff3-b02d-cdc64b50cdaa,K-05363070,60646431ae18cb00063ed63f,1862
10,c2626398-faac-4ff3-b02d-cdc64b50cdaa,K-01387534,60646431ae18cb00063ed63f,1862
...,...,...,...,...
548074,8b0915ff-a0cb-4520-9160-8d783a6bf308,K-02106319,6110c56241e21e000857ca77,131
548110,8b0915ff-a0cb-4520-9160-8d783a6bf308,K-20333371,6110c56241e21e000857ca77,131
548137,8b0915ff-a0cb-4520-9160-8d783a6bf308,K-07027249,6110c56241e21e000857ca77,131
548162,8b0915ff-a0cb-4520-9160-8d783a6bf308,K-02011774,6110c56241e21e000857ca77,131


In [19]:
# yhdistetään konserni-interaktiot tavallisiin ja poistetaan sellaiset interaktiot, joille ei löydy y-tunnusta
INTERACTIONS_DF = pd.concat([interactions_tmp, concern_interactions])
INTERACTIONS_DF = INTERACTIONS_DF[INTERACTIONS_DF.business_id.isin(ITEM_IDS)]
INTERACTIONS_DF[INTERACTIONS_DF['business_id'] == 'K-02011774']

,group_id,business_id,owner,group_size
18663,6c894b42-cbc6-4d18-8cf1-39ee91d2bf53,K-02011774,6058a712ae18cb00063ed639,2175
49204,66aea578-682d-45de-a200-77fa79a8c5e7,K-02011774,5db92c0ebc3e9100062ac0b0,3692
217786,608ac4ff-ab75-425b-a4f1-74d5defd43a3,K-02011774,5fbe6408f464de0006491d9e,1541
231830,4d2c2290-72be-40fe-8506-42718d2aac25,K-02011774,5f5613da9769490006b0ebb3,3685
254761,58f4cb31-0d40-42b2-9268-10d6dc64f1a0,K-02011774,5f5613da9769490006b0ebb3,2595
298418,2177a613-3b16-483e-9c9b-dc20bd4225f6,K-02011774,608863795602580007e70ddf,3925
354226,62c0017d-ff32-4584-8c85-562e9a1e8329,K-02011774,5fab9e2dc07a1900066bab26,3949
412123,3a44f374-56ee-4500-8829-c4b942b7afec,K-02011774,6033ae80484e8a0006fe437a,3966
439385,725e7cd0-7031-4988-b4cb-69672c611514,K-02011774,5db83a1cbc3e9100062ac0ab,1846
451491,acfdacd6-0de0-4568-a819-c3c2e15ef221,K-02011774,6033ae80484e8a0006fe437a,3809


## Poimitaan käytössä olevat ominaisuudet

In [20]:
features_tmp = [COMPANIES_DF[feature].unique() for feature in SELECTED_COMPANY_FEATURES]
FEATURES = [item for sublist in features_tmp for item in sublist]
len(FEATURES)
FEATURES

['company_form_code+CO_26',
 'company_form_code+CO_16',
 'company_form_code+CO_53',
 'company_form_code+CO_2',
 'company_form_code+CO_10',
 'company_form_code+CO_13',
 'company_form_code+CO_19',
 'company_form_code+CO_5',
 'company_form_code+CO_14',
 'company_form_code+CO_6',
 'company_form_code+CO_63',
 'company_form_code+CO_51',
 'company_form_code+CO_56',
 'company_form_code+CO_18',
 'company_form_code+CO_60',
 'company_form_code+CO_17',
 'company_form_code+CO_0',
 'company_form_code+CO_54',
 'company_form_code+CO_52',
 'company_form_code+CO_50',
 'company_form_code+CO_57',
 'company_form_code+CO_31',
 'company_form_code+CO_29',
 'company_form_code+CO_41',
 'company_form_code+CO_71',
 'company_form_code+CO_39',
 'company_form_code+CO_36',
 'company_form_code+CO_90',
 'company_form_code+CO_48',
 'company_form_code+CO_44',
 'company_form_code+CO_59',
 'company_form_code+CO_55',
 'company_form_code+CO_42',
 'company_form_code+CO_40',
 'company_form_code+CO_15',
 'company_form_code+CO_7

## Lasketaan mitkä ominaisuudet esiintyvät ryhmissä vallitsevina

In [21]:
def calculate_criteria_index_for_feature(features, threshold):
    results = [0 for feature in features]
    interaction_business_df = INTERACTIONS_DF.merge(COMPANIES_DF, on='business_id')
    group_ids = list(INTERACTIONS_DF['group_id'].unique())

    for gid in group_ids:
        group_df = interaction_business_df.loc[interaction_business_df['group_id'] == gid]
        num_companies = group_df.shape[0]

        for i, feature in enumerate(features):
            col_name = feature.split('+')[0]
            num_with_feature = group_df[group_df[col_name] == feature].shape[0]
            feature_percent = num_with_feature / num_companies
            if (feature_percent >= threshold):
                results[i] = results[i] + 1


            

    return pd.DataFrame({'feature': features, 'count_over_thresh': results})

        

In [22]:
RESULTS_DF = calculate_criteria_index_for_feature(FEATURES, 0.5)

In [24]:

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(RESULTS_DF.sort_values(['count_over_thresh', 'feature']))

                            feature  count_over_thresh
16           company_form_code+CO_0                  0
53           company_form_code+CO_1                  0
41          company_form_code+CO_12                  0
34          company_form_code+CO_15                  0
6           company_form_code+CO_19                  0
3            company_form_code+CO_2                  0
58          company_form_code+CO_20                  0
48          company_form_code+CO_21                  0
50          company_form_code+CO_22                  0
64          company_form_code+CO_23                  0
42          company_form_code+CO_24                  0
55          company_form_code+CO_25                  0
51          company_form_code+CO_28                  0
22          company_form_code+CO_29                  0
56           company_form_code+CO_3                  0
47          company_form_code+CO_30                  0
21          company_form_code+CO_31                  0
37        

## Poistetaan sijaintikunnat ylläolevan perusteella

In [25]:
SELECTED_COMPANY_FEATURES = ['company_form_code', 
                             'location_region_code', 'company_status_code', 'industry_code', 'turnover', 
                             'net_profit', 'personnel_average', 'performer_ranking_points', 'risk_rating_class']

COMPANIES_DF = COMPANIES_DF[['business_id', 'company_name'] + SELECTED_COMPANY_FEATURES]
COMPANIES_DF.head()

,business_id,company_name,company_form_code,location_region_code,company_status_code,industry_code,turnover,net_profit,personnel_average,performer_ranking_points,risk_rating_class
0,31431209,Tmi Onerva Miettunen,company_form_code+CO_26,location_region_code+01,company_status_code+AKT,industry_code+43,turnover+NaN,net_profit+NaN,personnel_average+NaN,performer_ranking_points+NaN,risk_rating_class+NaN
1,32087307,Koti Vaihtoon LKV Oy,company_form_code+CO_16,location_region_code+01,company_status_code+AKT,industry_code+68,turnover+NaN,net_profit+NaN,personnel_average+NaN,performer_ranking_points+NaN,risk_rating_class+NaN
2,18601103,Teollisuuden Monialapalvelu T:mi Oksanen,company_form_code+CO_26,location_region_code+NaN,company_status_code+AKT,industry_code+68,turnover+NaN,net_profit+NaN,personnel_average+NaN,performer_ranking_points+NaN,risk_rating_class+NaN
3,20469041,POLYBEAT,company_form_code+CO_53,location_region_code+NaN,company_status_code+AKT,industry_code+74,turnover+NaN,net_profit+NaN,personnel_average+NaN,performer_ranking_points+NaN,risk_rating_class+NaN
4,24844507,SAHOJEN PUU,company_form_code+CO_26,location_region_code+NaN,company_status_code+AKT,industry_code+NaN,turnover+NaN,net_profit+NaN,personnel_average+NaN,performer_ranking_points+NaN,risk_rating_class+NaN


## Pikkelöi käsitelty yritysdata

In [ ]:
COMPANIES_DF.to_pickle(WORKING_DIRECTORY + "data/pandas_pickles/company_data_iteration3.pkl")